In [1]:
# connect
import psycopg2
import json
import numpy as np
import time
# more connectstr
# "dbname='sqlbook' user='postgres' host='localhost' password='xxxx'"

inputfile = open('connectstr', 'r')
connstr = None

for i in inputfile:
    connstr = json.loads(i.strip())
    
inputfile.close()

conn = psycopg2.connect(connstr)    
conn.set_session(autocommit = True)
cur = conn.cursor();

In [2]:
def extract_seasons(cursor):
    sql = """select asin,
          round(100*sum(case when month >= 3 and month < 6 then numunits else 0 end)/sum(numunits),2) as spring,
          round(100*sum(case when month >= 6 and month < 9 then numunits else 0 end)/sum(numunits),2) as summer,
          round(100*sum(case when month >= 9 and month < 12 then numunits else 0 end)/sum(numunits),2) as fall,
          round(100*sum(case when (month = 12 or month < 3) then numunits else 0 end)/sum(numunits),2) as winter 
          from 
          (select asin, EXTRACT(MONTH FROM orderdate) as month, case when l.numunits = 0 then 0.00001 else l.numunits end as numunits
          from customers c, orders o, orderlines l, products p
          where c.customerid = o.customerid
          and o.orderid = l.orderid
          and l.productid = p.productid
          ) as temp
          group by asin"""

    cursor.execute(sql)
    
    with open ('asin_seasons.txt', 'w') as seasons:
        for r in cursor.fetchall():
            seasons.write("{0}\n".format("\t".join(str(x) for x in r)))

In [3]:
extract_seasons(cur)